In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import scipy as sp
from math import sqrt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.read_excel('Nerlove1963.xlsx')
df['lnc'] = np.log(df['Cost'])
df['lnq'] = np.log(df['output'])
df['lnpl'] = np.log(df['Plabor'])
df['lnpk'] = np.log(df['Pcapital'])
df['lnpf'] = np.log(df['Pfuel'])
df['lnplkf'] = df['lnpl'] + df['lnpk'] + df['lnpf']
df['ic'] = 1

In [4]:
srtd_lnq = np.sort(df['lnq'])
gamma_l = srtd_lnq[15]
gamma_u = srtd_lnq[130]
print('Appropriate range for gamma: [', gamma_l, ', ', gamma_u, ']')

Appropriate range for gamma: [ 4.143134726391533 ,  8.66888370465667 ]


In [5]:
grid = np.linspace(gamma_l, gamma_u, num = 10000)

In [7]:
def itr(k):
  tmp = df[['lnc', 'ic', 'lnq', 'lnplkf']].copy() 
  tmp['gq'] = df['lnq']/(1+np.exp(-df['lnq']+grid[k])) 
  return tmp

In [9]:
searr = np.array([]) 
for i in range(10000):
    itri = itr(i)
    mdi = sm.OLS(itri['lnc'], itri[['ic', 'lnq', 'lnplkf', 'gq']])
    resi = mdi.fit(cov_type = 'HC2')
    searr = np.append(searr, resi.ssr/145)
gse = searr.min()
gammah = grid[searr.argmin()]
print(searr.argmin())
print('Gamma_hat is', gammah)

6036
Gamma_hat is 6.875150011200851


In [10]:
itr = itr(6036)
md = sm.OLS(itr['lnc'], itr[['ic', 'lnq', 'lnplkf', 'gq']])
res = md.fit(cov_type = 'HC2')
rsd = res.resid
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                     781.5
Date:                Wed, 08 Jun 2022   Prob (F-statistic):           1.29e-87
Time:                        15:57:21   Log-Likelihood:                -37.660
No. Observations:                 145   AIC:                             83.32
Df Residuals:                     141   BIC:                             95.23
Df Model:                           3                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -5.3211      0.513    -10.379      0.0

In [13]:
df1 = df[['lnc', 'ic', 'lnq', 'lnplkf']].copy()
df1['dbeta4'] = df['lnq']/(1+np.exp(-df['lnq']+gammah))
df1['dgamma'] = -res.params[3]*df['lnq']*(np.exp(-df['lnq']+gammah))/((1+np.exp(-df['lnq']+gammah))**2)
Gb = (df1[['ic', 'lnq', 'lnplkf', 'dbeta4', 'dgamma']].copy()).to_numpy()
qgg = Gb.T@Gb
omgh = np.diag(rsd)@np.diag(rsd)
vge = Gb.T@omgh@Gb
v = np.linalg.inv(qgg)@vge@np.linalg.inv(qgg)
print('Variance matrix is:', v)

Variance matrix is: [[ 0.24382005 -0.02895764 -0.01320376  0.0147946  -0.05959083]
 [-0.02895764  0.00991437 -0.00115884 -0.00535105  0.02704112]
 [-0.01320376 -0.00115884  0.00194905  0.00067614 -0.00339944]
 [ 0.0147946  -0.00535105  0.00067614  0.00297594 -0.0145128 ]
 [-0.05959083  0.02704112 -0.00339944 -0.0145128   0.12863013]]


In [14]:
print('Standard errors are', sqrt(v[0,0]), "for b1" ', ', sqrt(v[1,1]), "for b2" ', ', sqrt(v[2,2]), "for b3" ', ', sqrt(v[3,3
]),"for b4" ', ', sqrt(v[4,4]), ' for gamma')

Standard errors are 0.4937813767865487 for b1,  0.09957094932289591 for b2,  0.04414803932175496 for b3,  0.05455221811669976 for b4,  0.35865042525520147  for gamma


In [15]:
tmp1 = (1 + np.exp(-df['lnq'] + gammah) + df['lnq']*np.exp(-df['lnq'] + gammah))/((1 + np.exp(-df['lnq'] + gammah))**2)
avg_marg = np.sum(tmp1*res.params[3] + res.params[1])/145
lnqavg = np.mean(df['lnq'])
tmp2 = (1 + np.exp(-lnqavg + gammah) + lnqavg*np.exp(-lnqavg + gammah))/((1 + np.exp(-lnqavg + gammah))**2)
marg_avg = tmp2*res.params[3] + res.params[1]
print('Average marginal influence: ', avg_marg)
print('Marginal influence at means: ', marg_avg)

Average marginal influence:  0.8040957883539818
Marginal influence at means:  0.8901467101647086


In [16]:
def f(beta):
   return np.array(df['lnc'] - beta[0] - df['lnq']*beta[1] - df['lnplkf']*beta[2] - beta[3]*df['lnq']/(1 + np.exp(beta[4] - df['lnq']))) 
def g(dft, b0, b1, b2, b3, b4):
  return np.array(b0 + dft['lnq']*b1 + dft['lnplkf']*b2 + b3*dft['lnq']/(1 + np.exp(b4 - dft['lnq' ])))

In [17]:
beta0 = np.array([0,0,0,0,0])
beta1 = np.array([1,1,1,1,1])
beta01 = np.array([0,0,0,0,1])
beta10 = np.array([1,1,1,1,0])

In [18]:
nlls1 = sp.optimize.least_squares(f, beta0)

In [19]:
qgg1 = (nlls1.jac).T@nlls1.jac
print(qgg1)

[[  144.9999995    950.71440272  1310.40887363   948.67040225
     29.90501165]
 [  950.71440272  6760.36620118  8573.89576648  6753.79506675
     98.92817673]
 [ 1310.40887363  8573.89576648 11869.67847738  8555.11863327
    274.64086287]
 [  948.67040225  6753.79506675  8555.11863327  6747.3800864
     96.72376143]
 [   29.90501165    98.92817673   274.64086287    96.72376143
     31.22233724]]


In [20]:
betah1 = nlls1.x
print(betah1)

[ -7.6469326   16.34232246   0.34480276 -15.42140718  -0.7848473 ]


In [21]:
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac
print(vge1)

[[  12.93686258   53.63569827  118.79463142   52.86594121   11.05199091]
 [  53.63569827  297.70074731  488.95828595  296.04208248   24.29901653]
 [ 118.79463142  488.95828595 1092.20769353  481.78388321  102.98695209]
 [  52.86594121  296.04208248  481.78388321  294.46641522   23.12318001]
 [  11.05199091   24.29901653  102.98695209   23.12318001   16.6932328 ]]


In [22]:
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1) 
for i in range(5):
  print(sqrt(var1[i,i]))

0.48374221630189984
29.971823527933513
0.039198920823368524
29.98972813860967
2.237942711350128


In [23]:
print(var1)

[[ 2.34006532e-01  4.00548808e+00 -1.60192142e-02 -4.01688969e+00
  -3.53001691e-01]
 [ 4.00548808e+00  8.98310206e+02  4.99158816e-02 -8.98846448e+02
  -6.67623795e+01]
 [-1.60192142e-02  4.99158816e-02  1.53655539e-03 -4.96231979e-02
  -1.99906847e-03]
 [-4.01688969e+00 -8.98846448e+02 -4.96231980e-02  8.99383794e+02
   6.68072189e+01]
 [-3.53001691e-01 -6.67623795e+01 -1.99906848e-03  6.68072189e+01
   5.00838758e+00]]


In [24]:
nlls1 = sp.optimize.least_squares(f, beta1)
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.00000005   950.71440543  1310.40887233   948.67033706
     29.90504288]
 [  950.71440543  6760.36620177  8573.89572256  6753.79483423
     98.92848301]
 [ 1310.40887233  8573.89572256 11869.67841187  8555.11796648
    274.6411446 ]
 [  948.67033706  6753.79483423  8555.11796648  6747.37963048
     96.7240022 ]
 [   29.90504288    98.92848301   274.6411446     96.7240022
     31.22226018]]
Beta_hat:  [ -7.64693268  16.34186506   0.34480252 -15.42094951  -0.78481357]
Vge:  [[  12.93686264   53.63569912  118.79463746   52.86591748   11.05198869]
 [  53.63569912  297.70072438  488.95829393  296.04200479   24.29904134]
 [ 118.79463746  488.95829393 1092.20779759  481.78366153  102.98693924]
 [  52.86591748  296.04200479  481.78366153  294.46628797   23.12316898]
 [  11.05198869   24.29904134  102.98693924   23.12316898   16.69320482]]
sd_0:  0.4837403829186554
sd_1:  29.96989147882688
sd_2:  0.03919887487146824
sd_3:  29.98779593678886
sd_4:  2.2378740198730767
Covariance matr

In [25]:
nlls1 = sp.optimize.least_squares(f, beta01)
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.00000145   950.71440944  1310.40888567   948.67026513
     29.90505556]
 [  950.71440944  6760.36620066  8573.89577575  6753.79458047
     98.92864875]
 [ 1310.40888567  8573.89577575 11869.67853558  8555.11732184
    274.64126312]
 [  948.67026513  6753.79458047  8555.11732184  6747.37913524
     96.72409243]
 [   29.90505556    98.92864875   274.64126312    96.72409243
     31.22217114]]
Beta_hat:  [ -7.646936    16.34133361   0.34480256 -15.4204177   -0.78477429]
Vge:  [[  12.93686265   53.63569724  118.79464393   52.86588743   11.05198722]
 [  53.63569724  297.70065839  488.95828195  296.04187661   24.2990708 ]
 [ 118.79464393  488.95828195 1092.20791501  481.78338618  102.98693467]
 [  52.86588743  296.04187661  481.78338618  294.46610362   23.12315698]
 [  11.05198722   24.2990708   102.98693467   23.12315698   16.6931759 ]]
sd_0:  0.48374062562470205
sd_1:  29.96760775503192
sd_2:  0.03919888015925777
sd_3:  29.985512396708604
sd_4:  2.237792706751792
Covariance ma

In [26]:
nlls1 = sp.optimize.least_squares(f, beta10)
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  144.99999646   950.71439052  1310.40886308   948.67060083
     29.90489495]
 [  950.71439052  6760.36620711  8573.89578491  6753.7957631
     98.9272463 ]
 [ 1310.40886308  8573.89578491 11869.67853963  8555.1205881
    274.63981   ]
 [  948.67060083  6753.7957631   8555.1205881   6747.38144187
     96.72304526]
 [   29.90489495    98.9272463    274.63981       96.72304526
     31.2225096 ]]
Beta_hat:  [ -7.64692846  16.34378914   0.34480325 -15.4228749   -0.78495685]
Vge:  [[  12.93686269   53.63570004  118.79461493   52.86602217   11.05198287]
 [  53.63570004  297.70087639  488.95828589  296.0423865    24.29889126]
 [ 118.79461493  488.95828589 1092.20738564  481.78462316  102.98685541]
 [  52.86602217  296.0423865   481.78462316  294.46687733   23.1231721 ]
 [  11.05198287   24.29889126  102.98685541   23.1231721    16.69328591]]
sd_0:  0.48374173935874387
sd_1:  29.978383822806805
sd_2:  0.03919897586602662
sd_3:  29.99628879714104
sd_4:  2.238188314921182
Covariance matr

In [27]:
nlls1 = sp.optimize.least_squares(f, [-1,0.5,0.5,0.5,5])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  144.99999996   950.71440482  1310.40887829   533.88135372
    -37.47819154]
 [  950.71440482  6760.36620039  8573.89577488  4168.58397384
   -268.63337813]
 [ 1310.40887829  8573.89577488 11869.67852502  4803.88161668
   -337.26034082]
 [  533.88135372  4168.58397384  4803.88161668  2969.35832001
   -159.85791424]
 [  -37.47819154  -268.63337813  -337.26034082  -159.85791424
     12.01428521]]
Beta_hat:  [-5.32109613  0.43780691  0.37073415  0.22400569  6.87527291]
Vge:  [[ 1.42720738e+01  5.79533894e+01  1.31063109e+02  1.97329092e+01
  -1.51454836e+00]
 [ 5.79533894e+01  3.22254000e+02  5.28460725e+02  1.53053555e+02
  -9.88926121e+00]
 [ 1.31063109e+02  5.28460725e+02  1.20502562e+03  1.79032312e+02
  -1.36952435e+01]
 [ 1.97329092e+01  1.53053555e+02  1.79032312e+02  1.08906190e+02
  -4.90041776e+00]
 [-1.51454836e+00 -9.88926121e+00 -1.36952435e+01 -4.90041776e+00
   3.99380512e-01]]
sd_0:  0.4937730392422245
sd_1:  0.09956762913694774
sd_2:  0.044148093631029256
sd_3:  

In [28]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,5])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  144.99999978   950.71440389  1310.40887766   533.93952586
    -37.47917139]
 [  950.71440389  6760.36620048  8573.89577587  4169.0009373
   -268.63463501]
 [ 1310.40887766  8573.89577587 11869.67852875  4804.40510648
   -337.26948265]
 [  533.93952586  4169.0009373   4804.40510648  2969.85458877
   -159.86959419]
 [  -37.47917139  -268.63463501  -337.26948265  -159.86959419
     12.01470322]]
Beta_hat:  [-5.32108315  0.43777194  0.37074168  0.22402156  6.8749252 ]
Vge:  [[ 1.42720738e+01  5.79543926e+01  1.31063065e+02  1.97365712e+01
  -1.51475542e+00]
 [ 5.79543926e+01  3.22266399e+02  5.28470185e+02  1.53081187e+02
  -9.89027657e+00]
 [ 1.31063065e+02  5.28470185e+02  1.20502480e+03  1.79065967e+02
  -1.36971412e+01]
 [ 1.97365712e+01  1.53081187e+02  1.79065967e+02  1.08932415e+02
  -4.90111674e+00]
 [-1.51475542e+00 -9.89027657e+00 -1.36971412e+01 -4.90111674e+00
   3.99452054e-01]]
sd_0:  0.4937966189459736
sd_1:  0.09957701820788313
sd_2:  0.04414794108256301
sd_3:  0.

In [29]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,10])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  144.99999964   950.71440345  1310.40887685   533.92750137
    -37.47896922]
 [  950.71440345  6760.36620557  8573.89577672  4168.91475619
   -268.63437819]
 [ 1310.40887685  8573.89577672 11869.67852559  4804.29690423
   -337.2675965 ]
 [  533.92750137  4168.91475619  4804.29690423  2969.75200946
   -159.86718174]
 [  -37.47896922  -268.63437819  -337.2675965   -159.86718174
     12.01461705]]
Beta_hat:  [-5.32108585  0.43777916  0.37074012  0.22401828  6.87499707]
Vge:  [[ 1.42720738e+01  5.79541851e+01  1.31063074e+02  1.97358141e+01
  -1.51471263e+00]
 [ 5.79541851e+01  3.22263836e+02  5.28468229e+02  1.53075475e+02
  -9.89006679e+00]
 [ 1.31063074e+02  5.28468229e+02  1.20502497e+03  1.79059009e+02
  -1.36967491e+01]
 [ 1.97358141e+01  1.53075475e+02  1.79059009e+02  1.08926993e+02
  -4.90097228e+00]
 [-1.51471263e+00 -9.89006679e+00 -1.36967491e+01 -4.90097228e+00
   3.99437273e-01]]
sd_0:  0.4937917413646199
sd_1:  0.09957507726593319
sd_2:  0.044147971632270785
sd_3:  

In [30]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,15])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  144.9999996    950.71440339  1310.4088766    533.8865007
    -37.47827863]
 [  950.71440339  6760.36620103  8573.89577578  4168.62087375
   -268.63349253]
 [ 1310.4088766   8573.89577578 11869.67852392  4803.92794351
   -337.26115373]
 [  533.8865007   4168.62087375  4803.92794351  2969.40223734
   -159.8589497 ]
 [  -37.47827863  -268.63349253  -337.26115373  -159.8589497
     12.01432248]]
Beta_hat:  [-5.32109481  0.43780381  0.3707348   0.2240071   6.87524214]
Vge:  [[ 1.42720738e+01  5.79534783e+01  1.31063105e+02  1.97332334e+01
  -1.51456670e+00]
 [ 5.79534783e+01  3.22255098e+02  5.28461563e+02  1.53056002e+02
  -9.88935116e+00]
 [ 1.31063105e+02  5.28461563e+02  1.20502555e+03  1.79035291e+02
  -1.36954116e+01]
 [ 1.97332334e+01  1.53056002e+02  1.79035291e+02  1.08908512e+02
  -4.90047969e+00]
 [-1.51456670e+00 -9.88935116e+00 -1.36954116e+01 -4.90047969e+00
   3.99386849e-01]]
sd_0:  0.49377511534315466
sd_1:  0.09956846054359897
sd_2:  0.04414807958639056
sd_3:  0.

In [31]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,0])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.000002     950.71441381  1310.4088839    948.67021016
     29.90510364]
 [  950.71441381  6760.36620059  8573.89574207  6753.7943909
     98.92908602]
 [ 1310.4088839   8573.89574207 11869.67844771  8555.11674226
    274.64169573]
 [  948.67021016  6753.7943909   8555.11674226  6747.37876511
     96.72446915]
 [   29.90510364    98.92908602   274.64169573    96.72446915
     31.22210377]]
Beta_hat:  [ -7.64693725  16.3409126    0.34480241 -15.41999638  -0.78474308]
Vge:  [[  12.93686252   53.6356967   118.79464766   52.86586427   11.05198549]
 [  53.6356967   297.70062156  488.95827783  296.04178944   24.29909149]
 [ 118.79464766  488.95827783 1092.20799141  481.78317074  102.98692576]
 [  52.86586427  296.04178944  481.78317074  294.46597091   23.12314453]
 [  11.05198549   24.29909149  102.98692576   23.12314453   16.69315391]]
sd_0:  0.4837412947889851
sd_1:  29.96592920296398
sd_2:  0.03919879354801847
sd_3:  29.98383325757036
sd_4:  2.2377362965527174
Covariance matr

In [32]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,1])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.00000069   950.71440595  1310.40888585   948.67041625
     29.90500376]
 [  950.71440595  6760.36619899  8573.89580785  6753.79509727
     98.92808978]
 [ 1310.40888585  8573.89580785 11869.67861575  8555.11877362
    274.64079071]
 [  948.67041625  6753.79509727  8555.11877362  6747.38014793
     96.72368589]
 [   29.90500376    98.92808978   274.64079071    96.72368589
     31.22235214]]
Beta_hat:  [ -7.64693424  16.34240178   0.34480298 -15.42148654  -0.78485311]
Vge:  [[  12.93686274   53.63569701  118.79463122   52.86594437   11.05199109]
 [  53.63569701  297.70073117  488.95827057  296.0420759    24.29900356]
 [ 118.79463122  488.95827057 1092.20767733  481.78390909  102.98695219]
 [  52.86594437  296.0420759   481.78390909  294.46641725   23.12317344]
 [  11.05199109   24.29900356  102.98695219   23.12317344   16.69324122]]
sd_0:  0.4837432569414904
sd_1:  29.97239041750476
sd_2:  0.039198902552290985
sd_3:  29.99029541848231
sd_4:  2.2379728760683193
Covariance ma

In [34]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,25])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.           950.71440488  1310.40887868   533.87916296
    -37.47815437]
 [  950.71440488  6760.36620397  8573.8957774   4168.56827533
   -268.6333288 ]
 [ 1310.40887868  8573.8957774  11869.67852892  4803.8619061
   -337.25999409]
 [  533.87916296  4168.56827533  4803.8619061   2969.33963451
   -159.85747316]
 [  -37.47815437  -268.6333288   -337.25999409  -159.85747316
     12.01426929]]
Beta_hat:  [-5.32109664  0.43780823  0.37073387  0.2240051   6.875286  ]
Vge:  [[ 1.42720738e+01  5.79533515e+01  1.31063111e+02  1.97327712e+01
  -1.51454055e+00]
 [ 5.79533515e+01  3.22253533e+02  5.28460368e+02  1.53052515e+02
  -9.88922288e+00]
 [ 1.31063111e+02  5.28460368e+02  1.20502566e+03  1.79031045e+02
  -1.36951719e+01]
 [ 1.97327712e+01  1.53052515e+02  1.79031045e+02  1.08905203e+02
  -4.90039140e+00]
 [-1.51454055e+00 -9.88922288e+00 -1.36951719e+01 -4.90039140e+00
   3.99377812e-01]]
sd_0:  0.49377214569017774
sd_1:  0.09956727601944311
sd_2:  0.044148099064612936
sd_3:  

In [35]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,30])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.           950.71440533  1310.40887845   533.88646074
    -37.47827763]
 [  950.71440533  6760.36620821  8573.89577999  4168.62058447
   -268.63348899]
 [ 1310.40887845  8573.89577999 11869.67852454  4803.92757684
   -337.26114387]
 [  533.88646074  4168.62058447  4803.92757684  2969.40188943
   -159.8589399 ]
 [  -37.47827763  -268.63348899  -337.26114387  -159.8589399
     12.01432194]]
Beta_hat:  [-5.32109497  0.43780384  0.37073481  0.22400709  6.87524238]
Vge:  [[ 1.42720738e+01  5.79534774e+01  1.31063105e+02  1.97332306e+01
  -1.51456654e+00]
 [ 5.79534774e+01  3.22255088e+02  5.28461555e+02  1.53055981e+02
  -9.88935034e+00]
 [ 1.31063105e+02  5.28461555e+02  1.20502555e+03  1.79035266e+02
  -1.36954101e+01]
 [ 1.97332306e+01  1.53055981e+02  1.79035266e+02  1.08908492e+02
  -4.90047912e+00]
 [-1.51456654e+00 -9.88935034e+00 -1.36954101e+01 -4.90047912e+00
   3.99386792e-01]]
sd_0:  0.4937750997158138
sd_1:  0.0995684539144922
sd_2:  0.044148079720273556
sd_3:  0.

In [36]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,35])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.00000043   950.71440659  1310.40888043   533.87928636
    -37.47815674]
 [  950.71440659  6760.36620236  8573.89577746  4168.56915106
   -268.6333331 ]
 [ 1310.40888043  8573.89577746 11869.67852585  4803.86300731
   -337.2600154 ]
 [  533.87928636  4168.56915106  4803.86300731  2969.34067858
   -159.85749877]
 [  -37.47815674  -268.6333331   -337.2600154   -159.85749877
     12.01427032]]
Beta_hat:  [-5.32109658  0.43780815  0.37073388  0.22400513  6.87528527]
Vge:  [[ 1.42720737e+01  5.79533538e+01  1.31063110e+02  1.97327789e+01
  -1.51454099e+00]
 [ 5.79533538e+01  3.22253560e+02  5.28460390e+02  1.53052573e+02
  -9.88922511e+00]
 [ 1.31063110e+02  5.28460390e+02  1.20502565e+03  1.79031116e+02
  -1.36951760e+01]
 [ 1.97327789e+01  1.53052573e+02  1.79031116e+02  1.08905258e+02
  -4.90039291e+00]
 [-1.51454099e+00 -9.88922511e+00 -1.36951760e+01 -4.90039291e+00
   3.99377968e-01]]
sd_0:  0.4937721962650211
sd_1:  0.09956729583395312
sd_2:  0.0441480986554375
sd_3:  0.

In [37]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,40])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.           950.714405    1310.4088788    533.87857228
    -37.47814476]
 [  950.714405    6760.36620311  8573.89577906  4168.56404039
   -268.63331845]
 [ 1310.4088788   8573.89577906 11869.678531    4803.85659119
   -337.2599044 ]
 [  533.87857228  4168.56404039  4803.85659119  2969.33459458
   -159.85735606]
 [  -37.47814476  -268.63331845  -337.2599044   -159.85735606
     12.01426527]]
Beta_hat:  [-5.32109664  0.43780858  0.37073378  0.22400494  6.87528953]
Vge:  [[ 1.42720738e+01  5.79533416e+01  1.31063111e+02  1.97327342e+01
  -1.51453846e+00]
 [ 5.79533416e+01  3.22253409e+02  5.28460274e+02  1.53052236e+02
  -9.88921268e+00]
 [ 1.31063111e+02  5.28460274e+02  1.20502566e+03  1.79030704e+02
  -1.36951528e+01]
 [ 1.97327342e+01  1.53052236e+02  1.79030704e+02  1.08904938e+02
  -4.90038438e+00]
 [-1.51453846e+00 -9.88921268e+00 -1.36951528e+01 -4.90038438e+00
   3.99377093e-01]]
sd_0:  0.49377191829826256
sd_1:  0.09956718042751155
sd_2:  0.04414810031790381
sd_3:  

In [38]:
nlls1 = sp.optimize.least_squares(f, [1,0.5,0.5,0.5,-10])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.00000036   950.71440598  1310.40887968   948.67017509
     29.9051004 ]
 [  950.71440598  6760.36619579  8573.89576952  6753.79429589
     98.92907781]
 [ 1310.40887968  8573.89576952 11869.67850844  8555.11652024
    274.64165421]
 [  948.67017509  6753.79429589  8555.11652024  6747.37858396
     96.72443332]
 [   29.9051004     98.92907781   274.64165421    96.72443332
     31.22207745]]
Beta_hat:  [ -7.64693812  16.34072222   0.3448024  -15.41980589  -0.7847289 ]
Vge:  [[  12.93686241   53.63569535  118.79464969   52.86585277   11.0519872 ]
 [  53.63569535  297.70060119  488.95828134  296.04174726   24.29910888]
 [ 118.79464969  488.95828134 1092.20803868  481.78307939  102.98694306]
 [  52.86585277  296.04174726  481.78307939  294.46590911   23.12314658]
 [  11.0519872    24.29910888  102.98694306   23.12314658   16.69314846]]
sd_0:  0.4837425901031521
sd_1:  29.965085710007063
sd_2:  0.0391988413977697
sd_3:  29.98298989421753
sd_4:  2.237705477412771
Covariance matr

In [39]:
nlls1 = sp.optimize.least_squares(f, [0,0,0,0,5])
qgg1 = (nlls1.jac).T@nlls1.jac
print('Qgg: ', qgg1)
betah1 = nlls1.x
print('Beta_hat: ', betah1)
vge1 = (nlls1.jac).T@np.diag(nlls1.fun)@np.diag(nlls1.fun)@nlls1.jac 
print('Vge: ', vge1)
var1 = np.linalg.inv(qgg1)@vge1@np.linalg.inv(qgg1)
for i in range(5):
    print('sd_' + str(i) + ': ', sqrt(var1[i,i]))
print('Covariance matrix: ', var1)
print('Beta_4 is gamma')

Qgg:  [[  145.00000031   950.71440645  1310.40887982   533.93852012
    -37.47915499]
 [  950.71440645  6760.36621159  8573.89578014  4168.9937258
   -268.63461697]
 [ 1310.40887982  8573.89578014 11869.67852399  4804.39604642
   -337.26932866]
 [  533.93852012  4168.9937258   4804.39604642  2969.84600007
   -159.86939419]
 [  -37.47915499  -268.63461697  -337.26932866  -159.86939419
     12.01469628]]
Beta_hat:  [-5.32108323  0.43777254  0.37074153  0.22402129  6.87493122]
Vge:  [[ 1.42720738e+01  5.79543754e+01  1.31063066e+02  1.97365079e+01
  -1.51475185e+00]
 [ 5.79543754e+01  3.22266187e+02  5.28470024e+02  1.53080710e+02
  -9.89025914e+00]
 [ 1.31063066e+02  5.28470024e+02  1.20502481e+03  1.79065386e+02
  -1.36971085e+01]
 [ 1.97365079e+01  1.53080710e+02  1.79065386e+02  1.08931963e+02
  -4.90110472e+00]
 [-1.51475185e+00 -9.89025914e+00 -1.36971085e+01 -4.90110472e+00
   3.99450825e-01]]
sd_0:  0.49379623464353867
sd_1:  0.09957685642231902
sd_2:  0.0441479436543184
sd_3:  0.

In [40]:
gamma_l = 0
gamma_u = 10
grid = np.linspace(gamma_l, gamma_u, num = 10000) 
def itr(k):
  tmp = df[['lnc', 'ic', 'lnq', 'lnplkf']].copy() 
  tmp['gq'] = df['lnq']/(1+np.exp(-df['lnq']+grid[k])) 
  return tmp

In [43]:
searr = np.array([]) 
for i in range(10000):
    itri = itr(i)
    mdi = sm.OLS(itri['lnc'], itri[['ic', 'lnq', 'lnplkf', 'gq']])
    resi = mdi.fit(cov_type = 'HC2')
    searr = np.append(searr, resi.ssr/145)
gse = searr.min()
gammah = grid[searr.argmin()]
print(searr.argmin())
print('Gamma_hat is', gammah)

0
Gamma_hat is 0.0


In [44]:
itr = itr(6036)
md = sm.OLS(itr['lnc'], itr[['ic', 'lnq', 'lnplkf', 'gq']])
res = md.fit(cov_type = 'HC2')
rsd = res.resid
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     646.4
Date:                Wed, 08 Jun 2022   Prob (F-statistic):           3.62e-82
Time:                        17:49:50   Log-Likelihood:                -40.065
No. Observations:                 145   AIC:                             88.13
Df Residuals:                     141   BIC:                             100.0
Df Model:                           3                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -5.2728      0.585     -9.007      0.0

In [45]:
df1 = df[['lnc', 'ic', 'lnq', 'lnplkf']].copy()
df1['dbeta4'] = df['lnq']/(1+np.exp(-df['lnq']+gammah))
df1['dgamma'] = -res.params[3]*df['lnq']*(np.exp(-df['lnq']+gammah))/((1+np.exp(-df['lnq']+gammah))**2)
Gb = (df1[['ic', 'lnq', 'lnplkf', 'dbeta4', 'dgamma']].copy()).to_numpy()
qgg = Gb.T@Gb
omgh = np.diag(rsd)@np.diag(rsd)
vge = Gb.T@omgh@Gb
v = np.linalg.inv(qgg)@vge@np.linalg.inv(qgg)
print('Variance matrix is:', v)

Variance matrix is: [[ 3.11923438e-01  1.64153261e+00 -2.00219937e-02 -1.65846786e+00
   9.62626719e+00]
 [ 1.64153261e+00  8.91073940e+01 -1.02225594e-02 -8.92947760e+01
   3.98855863e+02]
 [-2.00219937e-02 -1.02225594e-02  1.80996415e-03  1.07276933e-02
  -1.10134410e-01]
 [-1.65846786e+00 -8.92947760e+01  1.07276933e-02  8.94837321e+01
  -3.99907970e+02]
 [ 9.62626719e+00  3.98855863e+02 -1.10134410e-01 -3.99907970e+02
   1.83959157e+03]]


In [46]:
print('Standard errors are', sqrt(v[0,0]), "for b1" ', ', sqrt(v[1,1]), "for b2" ', ', sqrt(v[2,2]), "for b3" ', ', sqrt(v[3,3
]),"for b4" ', ', sqrt(v[4,4]), ' for gamma')

Standard errors are 0.5585010634909353 for b1,  9.439671287978475 for b2,  0.04254367347894709 for b3,  9.459584141509252 for b4,  42.8904601129052  for gamma


In [47]:
gamma_l = 6
gamma_u = 16
grid = np.linspace(gamma_l, gamma_u, num = 10000) 
def itr(k):
  tmp = df[['lnc', 'ic', 'lnq', 'lnplkf']].copy() 
  tmp['gq'] = df['lnq']/(1+np.exp(-df['lnq']+grid[k])) 
  return tmp

In [48]:
searr = np.array([]) 
for i in range(10000):
    itri = itr(i)
    mdi = sm.OLS(itri['lnc'], itri[['ic', 'lnq', 'lnplkf', 'gq']])
    resi = mdi.fit(cov_type = 'HC2')
    searr = np.append(searr, resi.ssr/145)
gse = searr.min()
gammah = grid[searr.argmin()]
print(searr.argmin())
print('Gamma_hat is', gammah)

875
Gamma_hat is 6.875087508750875


In [49]:
itr = itr(6036)
md = sm.OLS(itr['lnc'], itr[['ic', 'lnq', 'lnplkf', 'gq']])
res = md.fit(cov_type = 'HC2')
rsd = res.resid
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     779.7
Date:                Wed, 08 Jun 2022   Prob (F-statistic):           1.50e-87
Time:                        17:56:45   Log-Likelihood:                -49.504
No. Observations:                 145   AIC:                             107.0
Df Residuals:                     141   BIC:                             118.9
Df Model:                           3                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ic            -5.3996      0.470    -11.494      0.0

In [51]:
df1 = df[['lnc', 'ic', 'lnq', 'lnplkf']].copy()
df1['dbeta4'] = df['lnq']/(1+np.exp(-df['lnq']+gammah))
df1['dgamma'] = -res.params[3]*df['lnq']*(np.exp(-df['lnq']+gammah))/((1+np.exp(-df['lnq']+gammah))**2)
Gb = (df1[['ic', 'lnq', 'lnplkf', 'dbeta4', 'dgamma']].copy()).to_numpy()
qgg = Gb.T@Gb
omgh = np.diag(rsd)@np.diag(rsd)
vge = Gb.T@omgh@Gb
v = np.linalg.inv(qgg)@vge@np.linalg.inv(qgg)
print('Variance matrix is:', v)

Variance matrix is: [[ 0.23220761 -0.02841533 -0.01190499  0.0146854  -0.00603866]
 [-0.02841533  0.01243588 -0.00231437 -0.00676781  0.00441727]
 [-0.01190499 -0.00231437  0.00229916  0.00128722 -0.0009788 ]
 [ 0.0146854  -0.00676781  0.00128722  0.0037578  -0.00247845]
 [-0.00603866  0.00441727 -0.0009788  -0.00247845  0.00252455]]


In [52]:
print('Standard errors are', sqrt(v[0,0]), "for b1" ', ', sqrt(v[1,1]), "for b2" ', ', sqrt(v[2,2]), "for b3" ', ', sqrt(v[3,3
]),"for b4" ', ', sqrt(v[4,4]), ' for gamma')

Standard errors are 0.48187925335889414 for b1,  0.11151625475194613 for b2,  0.04794955445219166 for b3,  0.061300922431569996 for b4,  0.05024487662121092  for gamma


The results are identical only when γ has right starting value. When it is too small or too big the results differ. In case of 50, for example, it does not work. However, smooth function tolerate greater differences between true γ and starting one. With concentration method we need a larger partition. Hence, these two methods work but differently. 